### Instructions to process the multi-turn dialog dataset of 

Aknowledgements to: Wu, Zixiu and Balloccu, Simone and Kumar, Vivek and Helaoui, Rim and Reiter, Ehud and Reforgiato Recupero, Diego and Riboni, Daniele

This data first appeared in the paper: </br> 
Anno-MI: A Dataset of Expert-Annotated Counselling Dialogues in ICASSP 2022 - 2022 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP). </br>
URL: https://ieeexplore.ieee.org/document/9746035/;jsessionid=W0Qfv89lx5LjTNei3Khmb2sjFcebu3IPajU2spOhFNBob0rhWZb7!1808242117

In [8]:
import pandas as pd
from functools import reduce
from typing import Dict

# download the data from github
df = pd.read_csv("https://github.com/uccollab/AnnoMI/blob/main/AnnoMI-full.csv?raw=true")

In [62]:
def transform_to_node(text: str, role: str) -> Dict:
    return {"text": text, "role": role, "meta": {}, "replies": []}

In [66]:
threads = []

for transcript_id in df["transcript_id"].unique():
    transcript = df[df["transcript_id"] == transcript_id]
    # since we are using a reduce strategy, we must sort in the reverse order
    transcript = transcript.sort_values("timestamp", ascending=False)
    transcript_nodes = transcript.apply(lambda x: transform_to_node(x["utterance_text"], x["interlocutor"]), axis=1)
    thread = {
        "thread": reduce(
            lambda x, y: {"text": y["text"], "role": y["role"], "meta": {}, "replies": [x]},  # therapist or client
            transcript_nodes.values,
        ),
        "source": f'Transcript of {transcript["video_url"].iloc[0]} extracted from https://www.youtube.com/watch?v=PaSKcfTmFEk',
        "meta": {},
    }
    threads.append(thread)